# Session 13: Visualization

In [ ]:
import logging
from datetime import datetime

import matplotlib.pyplot as plt
from pdgraster import RasterTiler

import parsl
from parsl import python_app
from parsl.config import Config
from parsl.channels import LocalChannel
from parsl.executors import HighThroughputExecutor
from parsl.providers import LocalProvider

# set up plotting in notebook
plt.style.use('seaborn-notebook')
font_size = 4
plt.rcParams['font.size'] = font_size
plt.rcParams['axes.labelsize'] = font_size
plt.rcParams['axes.titlesize'] = font_size
plt.rcParams['xtick.labelsize'] = font_size
plt.rcParams['ytick.labelsize'] = font_size
plt.rcParams['legend.fontsize'] = font_size
plt.rcParams['figure.titlesize'] = font_size
plt.rcParams['figure.dpi'] = 200

In session 8 and 11, tiles were created at the maximum zoom level (z: 13). To show this data in a performant way on a web map, we need to create lower resolution tiles, and they need to be in a web-accessible format (most browsers can't render GeoTIFF files, PNG is better.) - [Here is a good explanation](https://www.maptiler.com/google-maps-coordinates-tile-bounds-projection/)

In [ ]:
# Start with the same config as Session 11, and initialize a RasterTiler again
iwp_config = {
    'dir_input': 'example-data',
    'ext_input': '.gpkg',
    'dir_geotiff': 'geotiff',
    'dir_web_tiles': 'web_tiles',
    'dir_staged': 'staged',
    'simplify_tolerance': None,
    'statistics': [
        {
            'name': 'number_IWP_per_pixel',
            'weight_by': 'count',
            'property': 'centroids_per_pixel',
            'aggregation_method': 'sum',
            'resampling_method': 'sum',
            'palette': ['rgb(102 51 153 / 0.0)', '#d93fce', 'lch(85% 100 85)'],
            'val_range': [0, None]
        },
        {
            'name': 'prop_pixel_covered_by_IWP',
            'weight_by': 'area',
            'property': 'area_per_pixel_area',
            'aggregation_method': 'sum',
            'resampling_method': 'average',
            'palette': ['rgb(102 51 153 / 0.0)', 'lch(85% 100 85)'],
            'val_range': [0, 1]
        }
    ],
}
iwp_rasterizer = RasterTiler(iwp_config)

In [ ]:
# Set up Parsl and logging again:
activate_conda = 'source /home/thiessenbock/.bashrc; conda activate pdg'
htex_local = Config(
    executors=[
        HighThroughputExecutor(
            label="htex_local",
            worker_debug=False,
            cores_per_worker=1,
            max_workers=26,
            provider=LocalProvider(
                channel=LocalChannel(),
                init_blocks=1,
                max_blocks=20,
                worker_init=activate_conda
            )
        )
    ],
)
parsl.clear()
parsl.load(htex_local)

logging.basicConfig(level=logging.INFO)

Here are the methods that we will use from the RasterTiler:

In [ ]:
# Which are the min and max z-levels we want to create tiles for?
help(iwp_rasterizer.config.get_min_z)
help(iwp_rasterizer.config.get_max_z)

In [ ]:
# Given a tile, identify the tile that is one z-level above it (lower resolution)
help(iwp_rasterizer.tiles.get_parent_tile)

In [ ]:
# Make a parent GeoTIFF tile from the four child tiles
help(iwp_rasterizer.parent_geotiffs_from_children)

In [ ]:
# This needs to be run before making webtiles from GeoTIFF files
help(iwp_rasterizer.update_ranges)

In [ ]:
# Apply a color map to a GeoTIFF and save it as a PNG file, using the same
# z/x/y directory structure
help(iwp_rasterizer.webtiles_from_geotiffs)

In [ ]:
# We'll also use the make_batch definition from Session 11 to create batches of
# GeoTIFF files to process.
def make_batch(items, batch_size):
    """
    Create batches of a given size from a list of items.
    """
    return [items[i:i + batch_size] for i in range(0, len(items), batch_size)]

In [ ]:
# Here are the parallel tasks we want to run:

@python_app
def create_composite_geotiffs(tiles, config):
    """
    Make composite geotiffs
    """
    import pdgraster
    import logging.config
    logging.basicConfig(level=logging.INFO)
    rasterizer = pdgraster.RasterTiler(config)
    return rasterizer.parent_geotiffs_from_children(tiles, recursive=False)

@python_app
def create_web_tiles(geotiff_paths, config, logging_dict=None):
    """
    Create a batch of webtiles from geotiffs (step 4)
    """
    import pdgraster
    import logging.config
    logging.basicConfig(level=logging.INFO)
    rasterizer = pdgraster.RasterTiler(config)
    return rasterizer.webtiles_from_geotiffs(
        geotiff_paths, update_ranges=False)

To create lower resolution GeoTIFF files, we can combine high resolution GeoTIFFs then resample them so that we still have 256x256 pixel data.

In [ ]:
# Get each z-level of GeoTIFFs we need to create:
min_z = iwp_rasterizer.config.get_min_z()
max_z = iwp_rasterizer.config.get_max_z()
parent_zs = list(range(max_z - 1, min_z - 1, -1))
print(f'Parent Zs: {parent_zs}')

## Create composite GeoTIFFs

In [ ]:
# Making composite GeoTIFFs is super fast, so let's choose a larger batch size
composite_geotiff_batch_size = 100

start_time = datetime.now()

# Can't start lower z-level until higher z-level is complete, because tiles in
# z-level n are created from tiles in z-level n+1, e.g. each level 12 tile is
# created from four level 13 tiles.
for z in parent_zs:

    # Determine which tiles we need to make for the next z-level based on
    # the path names of the files just created

    # Get the list of files that exist for the one-step-higher-resolution
    # z-level than the current z-level
    child_paths = iwp_rasterizer.tiles.get_filenames_from_dir('geotiff', z=z + 1)

    # Make a list of all the parent tiles that can be created from the z+1 child
    # tiles
    parent_tiles = set()
    for child_path in child_paths:
        parent_tile =  iwp_rasterizer.tiles.get_parent_tile(child_path)
        parent_tiles.add(parent_tile)
    parent_tiles = list(parent_tiles)

    # Break all parent tiles at level z into batches
    parent_tile_batches = make_batch(parent_tiles, composite_geotiff_batch_size)

    # Make parent GeoTIFFs for the current z-level (by combining the children tiles)
    app_futures = []
    for parent_tile_batch in parent_tile_batches:
        app_future = create_composite_geotiffs(
            parent_tile_batch, iwp_config)
        app_futures.append(app_future)

    # Don't start the next z-level, and don't move to step 4, until the
    # current z-level is complete
    [app_future.result() for app_future in app_futures]

end_time = datetime.now()
total_minutes = round(((end_time - start_time).total_seconds() / 60), 2)
print(f'🎉🎉🎉 Created composite GeoTIFF for {len(parent_zs)} z-levels in {total_minutes} minutes.')

# Make webtiles

In [ ]:
# Making webtiles is also super fast, so we can big a larger batch size
batch_size_web_tiles = 200

start_time = datetime.now()

# Get the min & max pixel values that exist across each z-level. Save these to
# the config file.
iwp_rasterizer.update_ranges()

# Process web tiles in batches
geotiff_paths = iwp_rasterizer.tiles.get_filenames_from_dir('geotiff')
geotiff_batches = make_batch(geotiff_paths, batch_size_web_tiles)

app_futures = []
for batch in geotiff_batches:
    app_future = create_web_tiles(batch, iwp_config)
    app_futures.append(app_future)

# Don't record end time until all web tiles have been created
[app_future.result() for app_future in app_futures]

end_time = datetime.now()

end_time = datetime.now()
total_minutes = round(((end_time - start_time).total_seconds() / 60), 2)
print(f'🎉🎉🎉 Converted all the GeoTIFFs to webtiles in {total_minutes} minutes.')

htex_local.executors[0].shutdown()
parsl.clear()
